# Backpack Prediction Challenge

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as plt
%matplotlib inline

In [56]:
train_datasets = pd.read_csv("../Data/train.csv")
test_datasets = pd.read_csv("../Data/test.csv")


In [ ]:
# train_datasets.info()
# train_datasets.isnull()

# test_datasets.isnull().sum()
# train_datasets['Brand'].unique()
# train_datasets['Material'].unique()
# train_datasets['Size'].unique()
# train_datasets['Laptop Compartment'].unique()
# train_datasets['Waterproof'].unique()
# train_datasets['Style'].unique()
# train_datasets['Color'].unique()
# train_datasets['Weight Capacity (kg)'].unique()

# train_datasets['Color'].mode()[0]
# train_datasets['Weight Capacity (kg)'].mean()


array(['Medium', 'Small', 'Large', nan], dtype=object)

In [58]:
# dealing with numerical columns, it might be better to use median (for robustness against outliers) instead of mean.

def fill_na(columns, type):
    if type == 'categorical':
        for col in columns:
            train_datasets[col] = train_datasets[col].fillna(train_datasets[col].mode()[0])
            test_datasets[col] = test_datasets[col].fillna(test_datasets[col].mode()[0])
    else:
        for col in columns:
            if col != 'Price':
                train_datasets[col] = train_datasets[col].fillna(train_datasets[col].median())
                test_datasets[col] = test_datasets[col].fillna(test_datasets[col].median())

In [59]:
# fill_na()

cat_cols = [col for col in train_datasets.columns if train_datasets[col].dtype == 'object']
num_cols = [col for col in train_datasets.columns if train_datasets[col].dtype != 'object']

fill_na(columns=cat_cols, type='categorical')
fill_na(columns=num_cols, type='numerical')

In [61]:
train_datasets['Laptop Compartment'] = train_datasets['Laptop Compartment'].map({'Yes': 1, 'No': 0})
train_datasets['Waterproof'] = train_datasets['Waterproof'].map({'Yes': 1, 'No': 0})
test_datasets['Laptop Compartment'] = test_datasets['Laptop Compartment'].map({'Yes': 1, 'No': 0})
test_datasets['Waterproof'] = test_datasets['Waterproof'].map({'Yes': 1, 'No': 0})

In [62]:
train_datasets.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,1,0,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,1,1,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,1,0,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,1,0,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,1,1,Messenger,Green,17.749338,86.02312


In [68]:
# train_datasets.to_csv('../Data/cleaned_train_data.csv', index=False)
test_datasets.to_csv('../Data/cleaned_test_data.csv', index=False)

In [65]:
print(train_datasets['Brand'].unique())
print(train_datasets['Material'].unique())
print(train_datasets['Size'].unique())
print(train_datasets['Style'].unique())
print(train_datasets['Color'].unique())

['Jansport' 'Under Armour' 'Nike' 'Adidas' 'Puma']
['Leather' 'Canvas' 'Nylon' 'Polyester']
['Medium' 'Small' 'Large']
['Tote' 'Messenger' 'Backpack']
['Black' 'Green' 'Red' 'Blue' 'Gray' 'Pink']


In [70]:
# create pipeline for encoding the data
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

categorical_cols_nominal = ['Brand', 'Material', 'Style', 'Color']  # Nominal categorical features
categorical_cols_ordinal = ['Size']  # Ordinal categorical features


# encoding the cat cols
transformer = ColumnTransformer(
    [
        ('nominal_cols_encoding', OneHotEncoder(), categorical_cols_nominal),
        ('ordinal_cols_encoding', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder(categories=[['Small', 'Medium', 'Large']])),
        ]), categorical_cols_ordinal)
    ]
)

# Create a pipeline to preprocess the data
pipeline = Pipeline(
    steps=[
        ('preprocessor', transformer)
    ]
)

# Fit and transform the training data and test data
train_datasets_transformed = pipeline.fit_transform(train_datasets)
test_datasets_transformed = pipeline.transform(test_datasets)


In [ ]:
train_datasets_transformed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1407144 stored elements and shape (300000, 19)>